In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pickle
from pathlib import Path
path = Path(r'C:\Users\matrix\Desktop\kaggle\芒果tv\Data_A')
import json



In [32]:

class MultiTaskDataset(Dataset):
    def __init__(self, df, cat_cols, num_cols, ctr_col=None, playtime_col=None, 
                 encoders=None, scaler=None, is_test=False):
        self.cat_cols = cat_cols
        self.num_cols = num_cols
        self.is_test = is_test
        
        # 处理类别型特征 按照训练集的情况再转一次
        self.cat_data = torch.tensor(
                df[cat_cols].to_numpy(), dtype=torch.long
            )
        # if encoders:
        #     self.cat_data = []
        #     for col in cat_cols:
        #         encoder = encoders[col]
        #         test_series = df[col].apply(lambda x: x if x in encoder.classes_ else -1)
        #         encoded = encoder.transform(test_series)
        #         self.cat_data.append(
        #             torch.tensor(encoded, dtype=torch.long)
        #         )
        #     self.cat_data = torch.stack(self.cat_data, dim=1)
        # else:
        #     self.encoders = {}
        #     self.cat_data = []
        #     for col in cat_cols:
        #         encoder = LabelEncoder()
        #         encoder.fit(list(df[col]) + [-1])
        #         self.encoders[col] = encoder
        #         encoded = encoder.transform(df[col])
        #         self.cat_data.append(torch.tensor(encoded, dtype=torch.long))
        #     self.cat_data = torch.stack(self.cat_data, dim=1)

        # 处理数值型特征 直接使用
        self.num_data = torch.tensor(
                df[num_cols].to_numpy(), dtype=torch.float32
            )
       
        # 处理标签
        if not is_test:
            self.ctr_labels = torch.tensor(df[ctr_col].values, dtype=torch.float32)
            self.playtime_labels = torch.tensor(df[playtime_col].values, dtype=torch.float32)
    
    def __len__(self):
        return len(self.cat_data)
    
    def __getitem__(self, idx):
        if self.is_test:
            return {
                'cat_features': self.cat_data[idx],
                'num_features': self.num_data[idx]
            }
        else:
            return {
                'cat_features': self.cat_data[idx],
                'num_features': self.num_data[idx],
                'ctr_label': self.ctr_labels[idx],
                'playtime_label': self.playtime_labels[idx]
            }


In [3]:

class MultiTaskDataset_t(Dataset):
    def __init__(self, df, cat_cols, num_cols, ctr_col=None, playtime_col=None, 
                 encoders=None, scaler=None, is_test=False):
        self.cat_cols = cat_cols
        self.num_cols = num_cols
        self.is_test = is_test
        
        # 处理类别型特征 按照训练集的情况再转一次
        if encoders:
            self.cat_data = []
            for col in cat_cols:
                encoder = encoders[col]
                test_series = df[col].apply(lambda x: x if x in encoder.classes_ else -1)
                encoded = encoder.transform(test_series)
                self.cat_data.append(
                    torch.tensor(encoded, dtype=torch.long)
                )
            self.cat_data = torch.stack(self.cat_data, dim=1)
        else:
            self.encoders = {}
            self.cat_data = []
            for col in cat_cols:
                encoder = LabelEncoder()
                encoder.fit(list(df[col]) + [-1])
                self.encoders[col] = encoder
                encoded = encoder.transform(df[col])
                self.cat_data.append(torch.tensor(encoded, dtype=torch.long))
            self.cat_data = torch.stack(self.cat_data, dim=1)
        # 处理数值型特征 直接使用
        self.num_data = torch.tensor(
                df[num_cols].to_numpy(), dtype=torch.float32
            )
       
        # 处理标签
        if not is_test:
            self.ctr_labels = torch.tensor(df[ctr_col].values, dtype=torch.float32)
            self.playtime_labels = torch.tensor(df[playtime_col].values, dtype=torch.float32)
    
    def __len__(self):
        return len(self.cat_data)
    
    def __getitem__(self, idx):
        if self.is_test:
            return {
                'cat_features': self.cat_data[idx],
                'num_features': self.num_data[idx]
            }
        else:
            return {
                'cat_features': self.cat_data[idx],
                'num_features': self.num_data[idx],
                'ctr_label': self.ctr_labels[idx],
                'playtime_label': self.playtime_labels[idx]
            }


In [4]:

class MultiTaskCTRModel(nn.Module):
    """多任务点击率和播放时长预测模型"""
    def __init__(self, cat_dims, embed_dims=8, shared_hidden_dims=[128, 64], 
                 ctr_hidden_dims=[32], playtime_hidden_dims=[32], dropout=0.2):
        super().__init__()
        
        # 嵌入层 - 处理类别型特征
        self.embeddings = nn.ModuleList([
            nn.Embedding(num_embeddings=dim, embedding_dim=embed_dims)
            for dim in cat_dims
        ])
        
        # 计算输入维度
        total_embed_dim = len(cat_dims) * embed_dims
        total_input_dim = total_embed_dim + len(num_cols)
        
        # 共享底层特征提取层
        shared_layers = []
        prev_dim = total_input_dim
        for dim in shared_hidden_dims:
            shared_layers.append(nn.Linear(prev_dim, dim))
            shared_layers.append(nn.BatchNorm1d(dim))
            shared_layers.append(nn.ReLU())
            shared_layers.append(nn.Dropout(dropout))
            prev_dim = dim
        self.shared_layers = nn.Sequential(*shared_layers)
        
        # CTR预测分支
        ctr_layers = []
        ctr_prev_dim = shared_hidden_dims[-1] if shared_hidden_dims else total_input_dim
        for dim in ctr_hidden_dims:
            ctr_layers.append(nn.Linear(ctr_prev_dim, dim))
            ctr_layers.append(nn.BatchNorm1d(dim))
            ctr_layers.append(nn.ReLU())
            ctr_layers.append(nn.Dropout(dropout))
            ctr_prev_dim = dim
        ctr_layers.append(nn.Linear(ctr_prev_dim, 1))
        ctr_layers.append(nn.Sigmoid())  # CTR是二分类问题
        self.ctr_head = nn.Sequential(*ctr_layers)
        
        # 播放时长预测分支
        playtime_layers = []
        playtime_prev_dim = shared_hidden_dims[-1] if shared_hidden_dims else total_input_dim
        for dim in playtime_hidden_dims:
            playtime_layers.append(nn.Linear(playtime_prev_dim, dim))
            playtime_layers.append(nn.BatchNorm1d(dim))
            playtime_layers.append(nn.ReLU())
            playtime_layers.append(nn.Dropout(dropout))
            playtime_prev_dim = dim
        playtime_layers.append(nn.Linear(playtime_prev_dim, 1))
        playtime_layers.append(nn.Sigmoid())
        self.playtime_head = nn.Sequential(*playtime_layers)

    def forward(self, x_cat, x_num):
        # 处理嵌入层
        embeds = [self.embeddings[i](x_cat[:, i]) for i in range(len(self.embeddings))]
        x_embed = torch.cat(embeds, dim=1)
        
        # 拼接特征
        x = torch.cat([x_embed, x_num], dim=1)
        
        # 通过共享层
        shared_features = self.shared_layers(x)
        
        # 通过各自的任务头
        ctr_output = self.ctr_head(shared_features)
        playtime_output = self.playtime_head(shared_features)
        
        return ctr_output.squeeze(), playtime_output.squeeze()
    

In [5]:

def train_multi_task_model(model, train_loader, valid_loader, device, epochs=2, patience=3, 
                           ctr_weight=0.5, playtime_weight=0.5):
    """训练多任务模型"""
    # 定义损失函数
    ctr_criterion = nn.BCELoss()
    playtime_criterion = nn.L1Loss()
    
    # 定义优化器
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    best_loss = float('inf')
    early_stop_count = 0
    
    for epoch in range(epochs):
        # 训练阶段
        model.train()
        train_ctr_loss = 0.0
        train_playtime_loss = 0.0
        train_total_loss = 0.0
        
        for batch in train_loader:
            x_cat = batch['cat_features'].to(device)
            x_num = batch['num_features'].to(device)
            ctr_y = batch['ctr_label'].to(device)
            playtime_y = batch['playtime_label'].to(device)
            
            optimizer.zero_grad()
            
            # 前向传播
            ctr_pred, playtime_pred = model(x_cat, x_num)
            
            # 计算损失
            ctr_loss = ctr_criterion(ctr_pred, ctr_y)
            playtime_loss = playtime_criterion(playtime_pred, playtime_y)
            total_loss = ctr_weight * ctr_loss + playtime_weight * playtime_loss
            print(f"ctr_loss:{ctr_loss},playtime_loss:{playtime_loss}")
            # 反向传播
            total_loss.backward()
            optimizer.step()
            
            # 累积损失
            train_ctr_loss += ctr_loss.item() * x_cat.size(0)
            train_playtime_loss += playtime_loss.item() * x_cat.size(0)
            train_total_loss += total_loss.item() * x_cat.size(0)
        
        # 计算平均损失
        train_ctr_loss /= len(train_loader.dataset)
        train_playtime_loss /= len(train_loader.dataset)
        train_total_loss /= len(train_loader.dataset)
        
        # 验证阶段
        model.eval()
        valid_ctr_loss = 0.0
        valid_playtime_loss = 0.0
        valid_total_loss = 0.0
        
        with torch.no_grad():
            for batch in valid_loader:
                x_cat = batch['cat_features'].to(device)
                x_num = batch['num_features'].to(device)
                ctr_y = batch['ctr_label'].to(device)
                playtime_y = batch['playtime_label'].to(device)
                
                ctr_pred, playtime_pred = model(x_cat, x_num)
                
                ctr_loss = ctr_criterion(ctr_pred, ctr_y)
                playtime_loss = playtime_criterion(playtime_pred, playtime_y)
                total_loss = ctr_weight * ctr_loss + playtime_weight * playtime_loss
                
                valid_ctr_loss += ctr_loss.item() * x_cat.size(0)
                valid_playtime_loss += playtime_loss.item() * x_cat.size(0)
                valid_total_loss += total_loss.item() * x_cat.size(0)
        
        # 计算平均损失
        valid_ctr_loss /= len(valid_loader.dataset)
        valid_playtime_loss /= len(valid_loader.dataset)
        valid_total_loss /= len(valid_loader.dataset)
        
        print(f'Epoch {epoch+1}/{epochs}')
        print(f'Train - CTR Loss: {train_ctr_loss:.4f}, Playtime Loss: {train_playtime_loss:.4f}, Total Loss: {train_total_loss:.4f}')
        print(f'Valid - CTR Loss: {valid_ctr_loss:.4f}, Playtime Loss: {valid_playtime_loss:.4f}, Total Loss: {valid_total_loss:.4f}')
        
        # 早停机制
        if valid_total_loss < best_loss:
            best_loss = valid_total_loss
            torch.save(model.state_dict(), 'best_multi_task_model.pth')
            early_stop_count = 0
        else:
            early_stop_count += 1
            if early_stop_count >= patience:
                print(f'Early stopping after {epoch+1} epochs')
                break
    
    # 加载最佳模型
    model.load_state_dict(torch.load('best_multi_task_model.pth'))
    return model


In [27]:
df_train = pd.read_parquet(r'C:\Users\matrix\Desktop\kaggle\芒果tv\code\ctr_df_train.parquet')
df_valid = pd.read_parquet(r'C:\Users\matrix\Desktop\kaggle\芒果tv\code\ctr_df_valid.parquet')
df_test = pd.read_parquet(r'C:\Users\matrix\Desktop\kaggle\芒果tv\code\ctr_test.parquet')
features = [c for c in df_train.columns if c not in ['click','play_time','play_ratio','day','did','vid','item_duration']]
num_cols = df_train[features].select_dtypes(include=['float16','float32','float64','int8','int16']).columns.tolist()
cat_cols = df_train[features].select_dtypes(exclude=['float16','float32','float64','int8','int16']).columns.tolist()
ctr_col = "click"
playtime_col = "play_ratio"

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')



Using device: cuda


In [28]:
for i in cat_cols:
    print(i)

item_cid
item_type
item_assetSource
item_classify
item_isIntact
item_serialno
sid
stype
f0
f1
f3
f5
f7
f9
f10
f12
f13
f15
f17
f19
f21
f23
f29
f31
f32
f34
f35
f37
f38
f40
f41
f43
f44
f46
f47
f48
f49
f51
f53
f55
f57
f58
f60
f61
f63
f64
f66
f67
f68
f69
f71
f72
f74
f76
f79
f80
f83
f84
f86


In [33]:

use_t = False
if use_t:
    # 创建数据集
    train_dataset = MultiTaskDataset_t(df_train, cat_cols, num_cols, ctr_col, playtime_col)
    valid_dataset = MultiTaskDataset_t(df_valid, cat_cols, num_cols, ctr_col, playtime_col,
                                    encoders=train_dataset.encoders)
    test_dataset = MultiTaskDataset_t(df_test, cat_cols, num_cols, None, None,
                                    encoders=train_dataset.encoders, is_test=True)
    cat_dims = [len(train_dataset.encoders[col].classes_) for col in cat_cols]
else:
    train_dataset = MultiTaskDataset(df_train, cat_cols, num_cols, ctr_col, playtime_col)
    valid_dataset = MultiTaskDataset(df_valid, cat_cols, num_cols, ctr_col, playtime_col)
    test_dataset = MultiTaskDataset(df_test, cat_cols, num_cols, None, None, is_test=True)
    cat_dims = []
    for col in cat_cols:
        with open(path/f'mappings/{col}_mapping.json', 'r') as f:
            mapping = json.load(f)
            cat_dims.append(len(mapping))
        

print(f"train_dataset: {len(train_dataset)}, valid_dataset: {len(valid_dataset)}, test_dataset: {len(test_dataset)}")
# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=4096, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=4096)
# test_loader = DataLoader(test_dataset, batch_size=4096)



train_dataset: 3978406, valid_dataset: 169139, test_dataset: 5162784


In [58]:
df_valid.head(5)

,did,vid,click,play_time,item_cid,item_type,item_duration,item_assetSource,item_classify,item_isIntact,...,f80,f81,f82,f83,f84,f85,f86,f87,day,play_ratio
0,bceccf13bdd2548bf19b120bed7dc78abb022dd4,23221081,True,26,2131,1,0.820497,0,1,1,...,47,0.020734,-0.016638,79921,553,-0.564113,11590,1.228537,8,31.688129
1,bceccf13bdd2548bf19b120bed7dc78abb022dd4,23202091,False,0,2101,1,1.903354,0,1,1,...,47,0.020734,-0.016638,79921,553,-0.564113,11590,1.228537,8,0.000000
2,bceccf13bdd2548bf19b120bed7dc78abb022dd4,23067005,False,0,2079,0,1.356075,0,1,1,...,47,0.020734,-0.016638,79921,553,-0.564113,11590,1.228537,8,0.000000
3,bceccf13bdd2548bf19b120bed7dc78abb022dd4,23205557,False,0,2133,0,1.102380,0,1,1,...,47,0.020734,-0.016638,79921,553,-0.564113,11590,1.228537,8,0.000000
4,bceccf13bdd2548bf19b120bed7dc78abb022dd4,23210843,False,0,2131,1,1.808684,0,1,1,...,47,0.020734,-0.016638,79921,553,-0.564113,11590,1.228537,8,0.000000


In [34]:

# 初始化模型
model = MultiTaskCTRModel(cat_dims).to(device)
print(f"start training")
# 训练模型
model = train_multi_task_model(model, train_loader, valid_loader, device)

start training
ctr_loss:0.7027392983436584,playtime_loss:0.4184807538986206
ctr_loss:0.69541335105896,playtime_loss:0.41686755418777466
ctr_loss:0.6831631064414978,playtime_loss:0.4134519100189209
ctr_loss:0.6806782484054565,playtime_loss:0.4079696834087372
ctr_loss:0.6704992651939392,playtime_loss:0.4037970304489136
ctr_loss:0.6669825315475464,playtime_loss:0.3994109630584717
ctr_loss:0.6610578298568726,playtime_loss:0.39800959825515747
ctr_loss:0.6549181938171387,playtime_loss:0.3941558003425598
ctr_loss:0.6467864513397217,playtime_loss:0.3894394040107727
ctr_loss:0.6414400339126587,playtime_loss:0.38643211126327515
ctr_loss:0.6332303285598755,playtime_loss:0.3837135136127472
ctr_loss:0.6273548007011414,playtime_loss:0.3780554533004761
ctr_loss:0.6276084780693054,playtime_loss:0.37228721380233765
ctr_loss:0.6188777685165405,playtime_loss:0.37183886766433716
ctr_loss:0.6106297969818115,playtime_loss:0.368053138256073
ctr_loss:0.6086199879646301,playtime_loss:0.3642340302467346
ctr_los